In [15]:
import pandas as pd
import numpy as np
from dateutil.parser import parse

In [16]:
exams_votes = pd.read_csv('../data/Esami aggiornati a ottobre 2017 per studenti progetto AAGM-DMO.csv')
anag_students = pd.read_csv('../data/Anagrafica per studenti progetto AAGM-DMO.csv')

#### Cambiamenti effettuati al dataset degli esami:
- riga 252 analisi 1 data a gennaio 2016 e lo studente si e' iscritto nel 2015 -> messo Giugno 2016
- riga 258 stessa cosa con l'esame di programmazione -> messo Giugno 2016
- riga 187 c'e' uno studente che ha dato analisi 1 a giugno 2015 e si e' iscritto nel 2015 -> messo Giugno 2016

# Data preprocessing

### Rimozione esami inglese e colonna giudizio e tipo

In [1]:
exams_votes = exams_votes[exams_votes['voto'] > 0]
del exams_votes['giudizio']
del exams_votes['tipo']

<IPython.core.display.Latex object>

In [17]:
exams_votes.head()

Id_studente codice_esame  data_esame tipo  voto giudizio  crediti  \
0           A      B006807  2015-01-16    S    24      NaN        6   
1           A      B006800  2014-07-17    S    29      NaN       12   
2           A      B006801  2014-06-10    S    30      NaN       12   
3           A      B006808  2017-01-23    S    18      NaN        6   
4           A      B006802  2014-06-12    S    25      NaN       12   

                                     descrizione  
0                                ALGEBRA LINEARE  
1                     ALGORITMI E STRUTTURE DATI  
2  ANALISI I: CALCOLO DIFFERENZIALE ED INTEGRALE  
3         ANALISI II: FUNZIONI DI PIU' VARIABILI  
4                 ARCHITETTURE DEGLI ELABORATORI

In [24]:
anag_students.head()

coorte Id_studente  test  voto_diploma tipo_diploma
0    2013           A    18            80           IT
1    2013           B    13            67           IT
2    2013           C    18            78           IT
3    2014           D    14            66           LS
4    2014           E     0            82           TC

### Parsing date tramite dateutil

In [25]:
def parse_dates_from_df():
    for date in list(exams_votes['data_esame']):
        if not isinstance(date, str) and np.isnan(date):
            yield None
        else:
            yield parse(date)
dates = list(parse_dates_from_df())

In [26]:
def to_str(datetime):
    return "{:04d}-{:02d}-{:02d}".format(datetime.year, datetime.month, datetime.day)

### Calcolo del semestre
Formula usata: ```((Anno_esame - Anno_coorte - 1) * 2 ( + 1 se l'esame e' fatto durante la sessione estiva))```

In [27]:
first_sem_exams_month = {1,2,3,4,5}
sec_sem_exams_month = {6,7,8,9,10,11,12}
def semester(datetime, coorte_year):
    year_diff = (datetime.year - coorte_year - 1) * 2
    if datetime.month in sec_sem_exams_month:
        year_diff += 1
    return year_diff

In [28]:
def an_stud_by_id(id_stud):
    return anag_students.loc[lambda l: l['Id_studente'] == id_stud]

### Calcolo dell'anno di coorte per ogni riga della tabella esami 

In [29]:
id_stud_col = exams_votes['Id_studente']
coorte_years = []
for id_stud in id_stud_col:
    coorte = int(an_stud_by_id(id_stud)['coorte']) 
    coorte_years.append(coorte)

### Applicazione del calcolo del semestre ed assegnazione alla tabella degli esami 

In [30]:
dates_coorte = list(zip(dates, coorte_years))
semesters = list(map(lambda pair: None if not all(pair) else semester(*pair), dates_coorte))
exams_votes['Semester'] = semesters

### Sostituzione dei semestri NaN

La strategia adottata consiste nel vedere quando gli altri studenti hanno svolto lo stesso esame ed usare la moda fra di essi per sostituire il valore nullo

In [31]:
rows_with_null_semsters = exams_votes[exams_votes['Semester'].isnull()]
for row in rows_with_null_semsters.itertuples():
    index_row, exam_id = row[0], row[2]
    exam_rows = exams_votes.loc[lambda df: df['codice_esame'] == exam_id]
    sem_mode = int(exam_rows['Semester'].mode())
    exams_votes.set_value(index_row, 'Semester', sem_mode) 

In [39]:
exams_votes['Semester'] = pd.to_numeric(exams_votes['Semester'], downcast='integer')

### Sostituzione delle date NaN in base al numero del semestre con una data di default

In [32]:
def calc_default_date(semester, coorte_year):
    first_semester = semester % 2 == 0
    year = coorte_year + 1 + (semester // 2)
    year = int(year)
    if first_semester:
        return f"15/01/{year}"
    else:
        return f"15/06/{year}"

In [33]:
rows_with_null_date = exams_votes[exams_votes['data_esame'].isnull()]
for row in rows_with_null_date.itertuples():
    anag_stud = an_stud_by_id(row.Id_studente)
    coorte = int(anag_stud.coorte)
    default_date = calc_default_date(row.Semester, coorte)
    exams_votes.set_value(row.Index, 'data_esame', default_date)

In [34]:
del rows_with_null_date
del rows_with_null_semsters

## Set dates with a unique format

In [35]:
dates_str = (to_str(date) for date in parse_dates_from_df())
exams_votes['data_esame'] = list(dates_str)

### Export in csv

In [43]:
exams_votes.to_csv('../data/exams_preproc.csv', sep=";", index=False)

In [42]:
?exams_votes.to_csv